<a href="https://colab.research.google.com/github/p25-c4/plate_project/blob/main/scale_train_test_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
df_raw = pd.read_csv("/content/drive/MyDrive/BigData/final_scale.csv")
df_raw.head()

,scale,spec_long,spec_country,steel_kind,pt_thick,pt_width,pt_length,hsb,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,day_yn,purpose
0,0,AB/EH32-TM,미국,T,32,3700,15100,적용,1,1,1144,116,1133,59,259,TMCP(온도제어),934,8,주간,선체구조용(조선용)
1,0,AB/EH32-TM,미국,T,32,3700,15100,적용,1,2,1144,122,1135,53,238,TMCP(온도제어),937,8,주간,선체구조용(조선용)
2,0,NV-E36-TM,영국,T,33,3600,19200,적용,2,1,1129,116,1121,55,258,TMCP(온도제어),889,8,주간,선체구조용(조선용)
3,0,NV-E36-TM,영국,T,33,3600,19200,적용,2,2,1152,125,1127,68,266,TMCP(온도제어),885,8,주간,선체구조용(조선용)
4,0,BV-EH36-TM,프랑스,T,38,3100,13300,적용,3,1,1140,134,1128,48,246,TMCP(온도제어),873,8,주간,선체구조용(조선용)


In [21]:
df_raw['descaling_count'] = df_raw['descaling_count'].apply(lambda x: 'odd' if x % 2 != 0 else str(x))  # 홀수는 'odd', 짝수는 문자열로 범주화

In [22]:
df_raw_x = df_raw.drop('scale', axis=1)
df_raw_x_1 = df_raw.drop(["scale", "spec_long",'steel_kind', 'day_yn', 'rolling_method'], axis=1)
df_raw_x_2 = df_raw[['steel_kind', 'day_yn', 'rolling_method']]  # 범주가 2개인 자료는 따로 dummy화
df_raw_y = df_raw['scale']

In [23]:

# 범주형 변수 선택 : select_dtypes="object"
df_raw_x_char_1 = df_raw_x_1.select_dtypes(include = "object")
df_raw_x_char_2 = df_raw_x_2.select_dtypes(include = "object")
df_raw_x_char = pd.concat([df_raw_x_char_1, df_raw_x_char_2], axis=1)

# get_dummies: 범주형 변수에 대한 더미변환
df_x_dummy_1 = pd.get_dummies(df_raw_x_char_1)
df_x_dummy_2 = pd.get_dummies(df_raw_x_char_2, drop_first=True) # 범주가 2개인 자료는 따로 dummy화
df_x_dummy = pd.concat([df_x_dummy_1, df_x_dummy_2], axis=1)


# 연속형 변수 선택 : select_dtypes="object" 제외
df_raw_x_num = df_raw_x.select_dtypes(exclude = "object")

# 변수명 저장
v_feature_names = df_raw_x_num.columns

# StandardScaler 적용
scaler = StandardScaler()
df_x_scaled = scaler.fit_transform(df_raw_x_num)
df_x_scaled = pd.DataFrame(df_x_scaled, columns=v_feature_names)

# 원래 연속형(df_raw_x_num) + dummy된 범주형(df_x_dummy)
df_x_raw_dummy = df_raw_x_num.join(df_x_dummy)
display("원래 연속형+ Dummied 범주형...DT/RF/GB 용:",df_x_raw_dummy.head(3))

# scale된 연속형(df_x_scaled)+ dummy된 범주형(df_x_dummy)
df_x_scale_dummy = df_x_scaled.join(df_x_dummy)
display("Scaled 연속형+ Dummied 범주형...SVM/NN/KNN 용:",df_x_scale_dummy.head(3))

# scale된 연속형(df_x_scaled) + 원래 범주형(df_raw_x_char)
df_x_scale_raw = df_x_scaled.join(df_raw_x_char)
display("Scaled 연속형+ 원래 범주형...회귀분석용(표준화 회귀계수):",df_x_scale_raw.head(3))

'원래 연속형+ Dummied 범주형...DT/RF/GB 용:'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,...,descaling_count_6,descaling_count_8,descaling_count_odd,purpose_OTHERS,purpose_선체구조용(조선용),purpose_용접구조용,purpose_일반구조용,steel_kind_T,day_yn_주간,rolling_method_TMCP(온도제어)
0,32,3700,15100,1,1,1144,116,1133,59,259,...,0,1,0,0,1,0,0,1,1,1
1,32,3700,15100,1,2,1144,122,1135,53,238,...,0,1,0,0,1,0,0,1,1,1
2,33,3600,19200,2,1,1129,116,1121,55,258,...,0,1,0,0,1,0,0,1,1,1


'Scaled 연속형+ Dummied 범주형...SVM/NN/KNN 용:'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,...,descaling_count_6,descaling_count_8,descaling_count_odd,purpose_OTHERS,purpose_선체구조용(조선용),purpose_용접구조용,purpose_일반구조용,steel_kind_T,day_yn_주간,rolling_method_TMCP(온도제어)
0,0.298839,1.765880,-1.561273,-1.227015,-0.982207,-0.623951,1.13104,-1.034421,-0.615879,0.534990,...,0,1,0,0,1,0,0,1,1,1
1,0.298839,1.765880,-1.561273,-1.227015,1.018115,-0.623951,1.35857,-0.918753,-0.906940,-0.017416,...,0,1,0,0,1,0,0,1,1,1
2,0.354957,1.562785,-1.265674,-0.003875,-0.982207,-1.333010,1.13104,-1.728433,-0.809920,0.508684,...,0,1,0,0,1,0,0,1,1,1


'Scaled 연속형+ 원래 범주형...회귀분석용(표준화 회귀계수):'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_temp,spec_country,hsb,descaling_count,purpose,steel_kind,day_yn,rolling_method
0,0.298839,1.765880,-1.561273,-1.227015,-0.982207,-0.623951,1.13104,-1.034421,-0.615879,0.534990,-0.103218,미국,적용,8,선체구조용(조선용),T,주간,TMCP(온도제어)
1,0.298839,1.765880,-1.561273,-1.227015,1.018115,-0.623951,1.35857,-0.918753,-0.906940,-0.017416,-0.056516,미국,적용,8,선체구조용(조선용),T,주간,TMCP(온도제어)
2,0.354957,1.562785,-1.265674,-0.003875,-0.982207,-1.333010,1.13104,-1.728433,-0.809920,0.508684,-0.803747,영국,적용,8,선체구조용(조선용),T,주간,TMCP(온도제어)


In [25]:
## train_test_split
# 원래 연속형+ Dummied 범주형...DT/RF/GB 용:df_x_raw_dummy
df_train_x_rd, df_test_x_rd, df_train_y, df_test_y = train_test_split(df_x_raw_dummy, df_raw_y
                                    , test_size = 0.2, stratify = df_x_raw_dummy["steel_kind_T"], random_state=1010)

display("원래 연속형+ Dummied 범주형...DT/RF/GB 용:", df_train_x_rd.head())

# 기계학습용 데이터(Scaled 연속형+ Dummied 범주형) 분할:df_x_scale_dummy
df_train_x_sd, df_test_x_sd, df_train_y, df_test_y = train_test_split(df_x_scale_dummy, df_raw_y
                                    , test_size = 0.2, stratify = df_x_raw_dummy["steel_kind_T"], random_state=1010)

display("Scaled 연속형+ Dummied 범주형...SVM/NN/KNN 용:", df_train_x_sd.head())

# 회귀분석용(표준화 회귀계수) 데이터(Scaled 연속형+ 원래 범주형) 분할:df_x_scale_raw
df_train_x_reg, df_test_x_reg, df_train_y, df_test_y = train_test_split(df_x_scale_raw, df_raw_y
                                    , test_size = 0.2, stratify = df_x_raw_dummy["steel_kind_T"], random_state=1010)

display("Scaled 연속형+ 원래 범주형...회귀분석용(표준화 회귀계수):",df_train_x_reg.head())

'원래 연속형+ Dummied 범주형...DT/RF/GB 용:'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,...,descaling_count_6,descaling_count_8,descaling_count_odd,purpose_OTHERS,purpose_선체구조용(조선용),purpose_용접구조용,purpose_일반구조용,steel_kind_T,day_yn_주간,rolling_method_TMCP(온도제어)
775,16,2600,49000,3,2,1177,76,1166,64,222,...,0,0,0,0,1,0,0,0,0,0
197,22,2200,36500,2,2,1164,71,1165,73,222,...,0,1,0,0,0,1,0,0,1,0
865,15,3100,51500,2,1,1192,76,1173,42,173,...,0,0,0,0,1,0,0,0,1,0
850,14,2800,43000,1,2,1130,118,1141,66,251,...,0,0,0,0,0,1,0,0,1,0
522,40,2500,22700,1,1,1178,90,1127,138,301,...,1,0,0,0,1,0,0,1,0,1


'Scaled 연속형+ Dummied 범주형...SVM/NN/KNN 용:'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,...,descaling_count_6,descaling_count_8,descaling_count_odd,purpose_OTHERS,purpose_선체구조용(조선용),purpose_용접구조용,purpose_일반구조용,steel_kind_T,day_yn_주간,rolling_method_TMCP(온도제어)
775,-0.599041,-0.468171,0.882821,1.219266,1.018115,0.935977,-0.385823,0.874110,-0.373329,-0.438297,...,0,0,0,0,1,0,0,0,0,0
197,-0.262336,-1.280553,-0.018394,-0.003875,1.018115,0.321460,-0.575431,0.816276,0.063263,-0.438297,...,0,1,0,0,0,1,0,0,1,0
865,-0.655158,0.547307,1.063064,-0.003875,-0.982207,1.645036,-0.385823,1.278950,-1.440552,-1.727244,...,0,0,0,0,1,0,0,0,1,0
850,-0.711276,-0.061980,0.450238,-1.227015,1.018115,-1.285739,1.206883,-0.571747,-0.276308,0.324549,...,0,0,0,0,0,1,0,0,1,0
522,0.747779,-0.671266,-1.013334,-1.227015,-0.982207,0.983248,0.145079,-1.381427,3.216424,1.639801,...,1,0,0,0,1,0,0,1,0,1


'Scaled 연속형+ 원래 범주형...회귀분석용(표준화 회귀계수):'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_temp,spec_country,hsb,descaling_count,purpose,steel_kind,day_yn,rolling_method
775,-0.599041,-0.468171,0.882821,1.219266,1.018115,0.935977,-0.385823,0.874110,-0.373329,-0.438297,0.566177,영국,적용,10,선체구조용(조선용),C,야간,CR(제어압연)
197,-0.262336,-1.280553,-0.018394,-0.003875,1.018115,0.321460,-0.575431,0.816276,0.063263,-0.438297,1.017629,한국,적용,8,용접구조용,C,주간,CR(제어압연)
865,-0.655158,0.547307,1.063064,-0.003875,-0.982207,1.645036,-0.385823,1.278950,-1.440552,-1.727244,0.239263,프랑스,적용,10,선체구조용(조선용),C,주간,CR(제어압연)
850,-0.711276,-0.061980,0.450238,-1.227015,1.018115,-1.285739,1.206883,-0.571747,-0.276308,0.324549,0.005753,일본,적용,10,용접구조용,C,주간,CR(제어압연)
522,0.747779,-0.671266,-1.013334,-1.227015,-0.982207,0.983248,0.145079,-1.381427,3.216424,1.639801,-3.045442,독일,적용,6,선체구조용(조선용),T,야간,TMCP(온도제어)
